In [3]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import numpy as np
from torch.utils.data import Dataset

class CharDataset(Dataset):

    def __init__(self, input_file):
        with open(input_file, 'r') as f:
            data = f.read()
        words = data.splitlines()
        words = [w.strip() for w in words] # get rid of any leading or trailing white space
        words = [w for w in words if w] # get rid of any empty strings
        chars = sorted(list(set(''.join(words)))) # all the possible characters
        chars.append('.')
        max_word_length = max(len(w) for w in words)
        print(f"number of examples in the dataset: {len(words)}")
        print(f"max word length: {max_word_length}")
        print(f"number of unique characters in the vocabulary: {len(chars)}")
        print("vocabulary:")
        print(''.join(chars))
        self.words = words
        self.chars = chars
        print("chars: {chars}")
        self.stoi = {ch:i for i,ch in enumerate(chars)}
        self.itos = {i:s for s,i in self.stoi.items()} 
        self.generate_tokens(input_file,max_word_length)
     

    def add_period_if_short(self,item):
        for i in range(17):
            if len(item) < i:
                item += '.'
        return item

   
    def generate_tokens(self,input_file,max_word_length):
        tokens=[]
        for item in self.words:
            wrd = self.add_period_if_short(item)
            tokens.extend([self.stoi[c] for c in wrd]) 
        # merged_tokens = torch.cat(all_tokens, dim=0)
        tokens_np = np.array(tokens)
        assert (0 <= tokens_np).all() and (tokens_np < 2**16).all(), "token dictionary too large for uint16"
        tokens_np_uint16 = tokens_np.astype(np.uint16)
        file_name = input_file + '_tokens'
        np.save(file_name, tokens_np_uint16)
    
    def __len__(self):
        return len(self.words)

    def contains(self, word):
        return word in self.words

    def get_vocab_size(self):
        return len(self.chars) + 1 # all the possible characters and special 0 token

    def encode(self, word):
        ix = torch.tensor([self.stoi[w] for w in word], dtype=torch.long)
        return ix

    def decode(self, ix):
        word = ''.join(self.itos[i] for i in ix)
        return word

    def __getitem__(self, idx):
        word = self.words[idx]
        ix = self.encode(word)
        tkns= torch.tensor(ix, dtype=torch.long)#x, y
        return tkns
    



In [ ]:
def load_tokens(filename):
    npt = np.load(filename)
    npt = npt.astype(np.int32) # added after video
    ptt = torch.tensor(npt, dtype=torch.long)
    return ptt

In [15]:
vocab='ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz.'
vocab =''.join(sorted(vocab))
vocab


'.ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz'

In [4]:

# # wrap in dataset objects
# file_name = 'babynames_train_0'
# CharDataset('./babynames/babynames.txt')
#token_filename = file_name + '_tokens.npy'
# ptt = load_tokens('./babynames/babynames_train_0.txt_tokens.npy')
# len(ptt)
CharDataset('./babynames/babynames_val.txt')

number of examples in the dataset: 271663
max word length: 15
number of unique characters in the vocabulary: 53
vocabulary:
ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz.
chars: {chars}


In [ ]:
6750857/32768

In [ ]:
import pandas as pd
# Read the Parquet file
df = pd.read_parquet('./raw_data/names_val.parquet')
# Select the specific column you want
column_data = df['Names']  # Replace 'your_column_name' with the actual column name
# Write the selected column to a text file
column_data.to_csv('babynames_val.txt', index=False, header=False)

In [ ]:
import numpy as np
with open('./babynames/babynames_val.txt', 'r') as f:
        data = f.read()
words = data.splitlines()
words = [w.strip() for w in words] # get rid of any leading or trailing white space
words = [w for w in words if w]
tokens = []
chars = sorted(list(set(''.join(words)))) 
stoi = {ch:i for i,ch in enumerate(chars)}
for item in words:
        tokens.extend([stoi[c] for c in item]) 
# merged_tokens = torch.cat(all_tokens, dim=0)
tokens_np = np.array(tokens)
assert (0 <= tokens_np).all() and (tokens_np < 2**16).all(), "token dictionary too large for uint16"
tokens_np_uint16 = tokens_np.astype(np.uint16)
file_name = 'babynames_val_' + '_tokens'
np.save(file_name, tokens_np_uint16)

In [ ]:
def fetch_until_next_uppercase(s):
    result = []
    temp = ''

    for char in s:
        if char.isupper() and temp:
            result.append(temp)
            temp = ''
        temp += char
    
    if temp:  # Append the last accumulated string if any
        result.append(temp)

    return result

# Example usage
input_string = "HelloWorldPythonIsAwesome"
segments = fetch_until_next_uppercase(input_string)
def add_period_if_short(s):
    for i in range(15):
        if len(s) < i:
            s += '.'
    return s

print("Segments:", segments)
for s in segments:
    print(add_period_if_short(s))


In [ ]:


# Example usage
file_path = 'babynames/babynames.txt' 
input_string = 
result = add_period_if_short(input_string)
print("Result:", result)


In [ ]:
def count_unique_names(file_path):
    unique_names = set()

    with open(file_path, 'r') as file:
        for line in file:
            name = line.strip()  # Remove leading/trailing whitespace, including newline characters
            if name:  # Ensure the line is not empty
                unique_names.add(name)
    
    return len(unique_names)

# Example usage
file_path = 'babynames/babynames.txt'  # Replace with the path to your file
unique_count = count_unique_names(file_path)
print(f"Number of unique names: {unique_count}")


In [ ]:
import numpy as np
import torch
npt = np.load('babynames/data_tokens/babynames_train_tokens.npy')
npt = npt.astype(np.int32) # added after video
ptt = torch.tensor(npt, dtype=torch.long)


In [ ]:
ptt.shape

In [ ]:
for i in range(16):
    print(i)

In [9]:
def add_period_if_short1(item):
    for i in range(16):
        if len(item) < i:
            item += '.'
    
    return item

name  =add_period_if_short1('ram')
name, len(name)

('ram............', 15)

In [5]:
import torch

checkpoint = torch.load("./log/model.pt", mmap=False)
# Initialize the model using the configuration saved in the checkpoint
model = GPT(checkpoint['config'])

# Load the saved state dict into the model
model.load_state_dict(checkpoint['model'])
model.to(device)
use_compile = False
# Optionally, retrieve the training step to continue from where it left off
trained_step = checkpoint['step']

RuntimeError: torch.UntypedStorage(): Storage device not recognized: mps